# Testing Multiple Models since our dataset is ready

In [54]:
import os 
import pandas as pd
import numpy as np
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers 
from tensorflow.keras import Model 
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical # used for converting labels to one-hot-encoding
from sklearn.model_selection import train_test_split
from glob import glob
from PIL import Image
import shutil


In [58]:
#Setting Global Variables
FINAL_PATH = "dataset_final//imagens//"

TRAIN_PATH = "dataset_final//train//"

TEST_PATH = "dataset_final//test//"

SIZE=224


In [87]:
pele_df = pd.read_csv('dataset_final/classificacao_dataset.csv')


le = LabelEncoder()
le.fit(pele_df['type'])
LabelEncoder()
print('label 0 é igual á ' + list(le.classes_)[0])
print('label 1 é igual á ' + list(le.classes_)[1])

pele_df['label'] = le.transform(pele_df["type"]) 


print(pele_df['label'].value_counts())

print(pele_df.head())

pele_df_final = pele_df.copy()

image_path = {os.path.splitext(os.path.basename(x))[0]: x
                     for x in glob(os.path.join('dataset_final', '*', '*.jpg'))}
#Define the path and add as a new column
pele_df_final['path'] = pele_df['image_id'].map(image_path.get)
#Use the path to read images.
#pele_df_final['imagem'] = pele_df_final['path'].map(lambda x: np.asarray(Image.open(x).resize((SIZE,SIZE))))


#Convert dataframe column of images into numpy array
X = pele_df_final['path']
Y = pele_df_final['type'] #Assign label values to Y
#Y_cat= to_categorical(Y, num_classes=2) #Convert to categorical as this is a multiclass classification problem

print(f'Unique values for Y: {list(set(Y))}')

x_train_auto, x_test_auto, y_train_auto, y_test_auto = train_test_split(X, Y, test_size=0.20, random_state=42)


print(f'x_train_auto: {x_train_auto[0:5]}')

print(f'x_test_auto: {x_test_auto[0:5]}')

print(f'y_train_auto: {y_train_auto[0:5]}')

print(f'y_test_auto: {y_test_auto[0:5]}')

label 0 é igual á mel
label 1 é igual á nv
1    6805
0    1183
Name: label, dtype: int64
   Unnamed: 0      image_id type  label
0           0  ISIC_0024698   nv      1
1           1  ISIC_0024693   nv      1
2           2  ISIC_0025964  mel      0
3           3  ISIC_0030623  mel      0
4           4  ISIC_0027190  mel      0
Unique values for Y: ['mel', 'nv']
x_train_auto: 7346    dataset_final/imagens/ISIC_0028596.jpg
3614    dataset_final/imagens/ISIC_0030647.jpg
1450    dataset_final/imagens/ISIC_0030540.jpg
7918          dataset_final/imagens/458192.jpg
6467    dataset_final/imagens/ISIC_0025240.jpg
Name: path, dtype: object
x_test_auto: 2748    dataset_final/imagens/ISIC_0032502.jpg
5583    dataset_final/imagens/ISIC_0033857.jpg
2572    dataset_final/imagens/ISIC_0030863.jpg
1954    dataset_final/imagens/ISIC_0030461.jpg
4727    dataset_final/imagens/ISIC_0032262.jpg
Name: path, dtype: object
y_train_auto: 7346    nv
3614    nv
1450    nv
7918    nv
6467    nv
Name: type, dtype:

In [89]:
df_train = pd.DataFrame(columns=['image_path','type'])
df_train['image_path'] = x_train_auto
df_train['type'] = y_train_auto

df_test = pd.DataFrame(columns=['image_path','type'])
df_test['image_path'] = x_test_auto
df_test['type'] = y_test_auto

In [91]:
#Creating a directory for Train 
for n, row in df_train.iterrows():
    if row['type']=='mel':
        shutil.copy2(row['image_path'], TRAIN_PATH + 'mel//')
    else: 
        shutil.copy2(row['image_path'], TRAIN_PATH+ 'nv//')

In [92]:
#Creating a directory for Test
for n, row in df_test.iterrows():
    if row['type']=='mel':
        shutil.copy2(row['image_path'], TEST_PATH + 'mel//')
    else: 
        shutil.copy2(row['image_path'], TEST_PATH+ 'nv//')

## Testing VGG-16

In [93]:
SIZE = 224

In [94]:
# Image Augmentation

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

In [95]:
# Training and Validation Sets
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(TRAIN_PATH, batch_size = 20, class_mode = 'binary', target_size = (224, 224))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(TEST_PATH,  batch_size = 20, class_mode = 'binary', target_size = (224, 224))

Found 6390 images belonging to 2 classes.
Found 1598 images belonging to 2 classes.


In [96]:
# Loading the Base Model
from keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape = (SIZE, SIZE, 3), # Shape of our images
include_top = True, # Leave out the last fully connected layer
weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False

Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/keras-applications/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels.h5: None -- [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)

In [ ]:
# Compile and Fit

# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer with 1 node for classification output
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001), loss = 'binary_crossentropy',metrics = ['acc'])

In [ ]:
vgghist = model.fit(train_generator, validation_data = validation_generator, steps_per_epoch = 100, epochs = 10)

## Testing Inception

In [97]:
SIZE = 150

In [98]:
# Data Augmentation
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255., rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2,shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

test_datagen = ImageDataGenerator( rescale = 1.0/255. )

In [99]:
# Training and Validation Generators

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(TRAIN_PATH, batch_size = 20, class_mode = 'binary', target_size = (224, 224))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(TEST_PATH,  batch_size = 20, class_mode = 'binary', target_size = (224, 224))

Found 6390 images belonging to 2 classes.
Found 1598 images belonging to 2 classes.


In [100]:
# Loading the Base Model
from tensorflow.keras.applications.inception_v3 import InceptionV3

base_model = InceptionV3(input_shape = (SIZE, SIZE, 3), include_top = False, weights = 'imagenet')

Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/keras-applications/inception_v3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5: None -- [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)

In [ ]:
# Compile and Fit

for layer in base_model.layers:
    layer.trainable = False

from tensorflow.keras.optimizers import RMSprop

x = layers.Flatten()(base_model.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)

# Add a final sigmoid layer with 1 node for classification output
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = RMSprop(lr=0.0001), loss = 'binary_crossentropy', metrics = ['acc'])

In [ ]:
inc_history = model.fit_generator(train_generator, validation_data = validation_generator, steps_per_epoch = 100, epochs = 10)

## Testing ResNet50

In [101]:
SIZE=224

In [102]:
# Data Augmentation and Generators
# Add our data-augmentation parameters to ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255., rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1.0/255.)

train_generator = train_datagen.flow_from_directory(TRAIN_PATH, batch_size = 20, class_mode = 'binary', target_size = (SIZE, SIZE))

validation_generator = test_datagen.flow_from_directory(TEST_PATH, batch_size = 20, class_mode = 'binary', target_size = (SIZE, SIZE))

Found 6390 images belonging to 2 classes.
Found 1598 images belonging to 2 classes.


In [103]:
#Import the base model
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")

for layer in base_model.layers:
    layer.trainable = False

Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/keras-applications/resnet/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5: None -- [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)

In [ ]:
# Build and Compile the Model
from tensorflow.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D

base_model = Sequential()
base_model.add(ResNet50(include_top=False, weights='imagenet', pooling='max'))
base_model.add(Dense(1, activation='sigmoid'))

In [ ]:
base_model.compile(optimizer = tf.keras.optimizers.SGD(lr=0.0001), loss = 'binary_crossentropy', metrics = ['acc'])

In [ ]:
resnet_history = base_model.fit(train_generator, validation_data = validation_generator, steps_per_epoch = 100, epochs = 10)

## Testing EfficientNet

In [ ]:
!pip install -U efficientnet

In [ ]:
import efficientnet.keras as efn

In [ ]:
SIZE = 224

In [ ]:
# Add our data-augmentation parameters to ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255., rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1.0/255.)

train_generator = train_datagen.flow_from_directory(train_dir, batch_size = 20, class_mode = 'binary', target_size = (224, 224))

validation_generator = test_datagen.flow_from_directory( validation_dir, batch_size = 20, class_mode = 'binary', target_size = (224, 224))
